In [ ]:
# Setup and Model Loading
import torch
from swift.utils import seed_everything
from modelscope import AutoModel, AutoTokenizer
from utils import load_image

model_path = 'HanXiao1999/DocMark-2B'
model = AutoModel.from_pretrained(
model_path,
torch_dtype=torch.bfloat16,
low_cpu_mem_usage=True,
use_flash_attn=True,
trust_remote_code=True).eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, use_fast=True, legacy=False, add_prefix_space=False)
generation_config = dict(max_new_tokens=2048, do_sample=False)
seed_everything(42)

InternLM2ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
[INFO:swift] Global seed set to 42


42

In [4]:
# Load and process text-rich natural image
# Extract text from the image
image_path = 'examples/text_img.jpg'
pixel_values = load_image(image_path, max_num=12).to(torch.bfloat16).cuda()
question = '<image>Could you extract the text from the image for me?'
answer = model.chat(tokenizer, pixel_values, question, generation_config, skip_special_tokens=False)
print('Answer:', answer)
# OCR with grounding
question = '<image>OCR with grounding:'
answer = model.chat(tokenizer, pixel_values, question, generation_config, skip_special_tokens=False)
print('Answer:', answer)

Answer: <txt>nutmegstate FEDERAL CREDIT UNION</txt>
Answer: <txt_gd><ref>nutmegstate</ref><quad>(334,104),(910,88),(912,200),(336,216)</quad><ref>FEDERAL CREDIT UNION</ref><quad>(336,228),(910,212),(911,252),(337,268)</quad><ref>nultmegstate</ref><quad>(212,524),(378,524),(378,552),(212,552)</quad> </txt_gd>


In [5]:
# Document Processing
# Convert document images into structured markup formats
image_path = 'examples/example_doc.png'
pixel_values = load_image(image_path, max_num=12).to(torch.bfloat16).cuda()
question = '<image>How can I convert this document into a structured markup language format?'
answer = model.chat(tokenizer, pixel_values, question, generation_config, skip_special_tokens=False)
print('Answer:', answer)


Answer: <md>---

Citizens' Commission on Science, Law and the Food Supply

Kindly have check drawn to order of

CHANNING H. LUSHBOUGH

to cover: REFUND OF PERSONAL LOAN AGAINST TRAVEL EXPENSES INCURRED BY DR. CRAMPTON

SEE VOUCHER #33 AND RECEIPT ATTACHED

Charge to
MEETINGS $100.00

APPROVED FOR PAYMENT

By C. X. Lushbough

---

Source: https://www.industrydocuments.ucsf.edu/docs/ljim0227</md>


In [6]:
# Document to LaTeX Conversion
image_path = 'examples/latex_page.jpg'
pixel_values = load_image(image_path, max_num=12).to(torch.bfloat16).cuda()
question = '<image>Convert the document into latex format:'
answer = model.chat(tokenizer, pixel_values, question, generation_config, skip_special_tokens=False)
print('Answer:', answer)

Answer: <latex>高数必练 1000 题答案解析由 (I) 知, 当 $n \rightarrow \infty$ 时, $x_{n} \rightarrow 0$, 故$\lim _{n \rightarrow \infty}\left(\frac{x_{n+1}}{x_{n}}\right)^{\frac{1}{x_{n}}}=\lim _{n \rightarrow \infty}\left(\frac{\sin x_{n}}{x_{n}}\right)^{\frac{1}{x_{n}}}=\lim _{n \rightarrow \infty}\left(\frac{\sin t}{t}\right)^{\frac{1}{t}}= e ^{-\frac{1}{t}}.$156.【证明】因$x_{n+1}=\sqrt{x_{n}\left(3-x_{n}\right)} \leqslant \sqrt{\left(\frac{x_{n}+3-x_{n}}{2}\right)^{\frac{2}{2}}}=\frac{3}{2}, n=1,2, \cdots$故 $\left\{x_{n}\right\}$ 有上界.又 $x_{n+1} \geqslant 0, x_{n+1}^{2}-x_{n}^{2}=x_{n}\left(3-x_{n}\right)-x_{n}^{2}=x_{n}\left(3-2 x_{n}\right) \geqslant 0$, 故 $x_{n+1} \geqslant x_{n}, n=$$2,3, \cdots$, 从而 $n>1$ 时, $\left\{x_{n}\right\}$ 为单调递增数列.由单调有界原理知, $\lim _{n \rightarrow \infty} x_{n}$ 存在, 不妨设 $A=\lim _{n \rightarrow \infty} x_{n}$, 由 $\lim _{n \rightarrow \infty} x_{n+1}=\lim _{n \rightarrow \infty}$$\sqrt{x_{n}\left(3-x_{n}\right)}$, 得 $A=\sqrt{A(3-A)}$, 即 $2 A^{2}=3 A$, 得 $A=\frac{3}{2}$ 或 $A=0$

In [7]:
# Webpage Analysis
# Analyze webpage screenshots to understand content, themes, and overall purpose
image_path = 'examples/example_webpage.png'
pixel_values = load_image(image_path, max_num=12).to(torch.bfloat16).cuda()
question = '<image>What is the central theme of the webpage picture?'
answer = model.chat(tokenizer, pixel_values, question, generation_config, skip_special_tokens=False)
print('Answer:', answer)

Answer: <html_brief>沁园雪，沁园雪源于韩国。表达了时尚、前卫及潮流个性的都市一族；以自由组合，浪漫休闲的色彩；追求约约、时尚的设计风格；高品质的面料，精美的工艺体现了“沁园雪” particular when ；设计风格以时尚、简洁、潮流、个性的表现形式完美包装都市女性，充分彰显现代女性的魅力和自信，简约设计，别致的细节变化，考究的面料，精巧的剪剪做工，更适合现代时尚女性的装扮需求。
沁园雪品牌含义
沁园雪品牌名称源于美国著名时尚家，做为时尚服装设计师的沁园雪同时喜欢登山，为著名的冒险家、体现了都市女性在城市和自然之间游刃有余追求个人自由，崇尚自然的精神。
沁园雪公司介绍
东莞海自服饰有限公司是一家集设计、开发、生产、销售为一体的女装企业，拥有自创“沁园雪”时装品牌。在产品上，公司投入了相当大的力度，招聘一批具有专业技术的产品设计队伍，并以设计为龙头，全力打造国际知名女装品牌为目标。
品牌优势
潮流：拥有专业的韩国设计师团队，解析最新的韩流趋势，从而研发出具有时尚、优雅、气质的款式。
实行：通过网络销售降低运营成本，真正的让销售商和消费者得到最大利益。
年龄定位：沁园雪”定位于18-35岁的都市白领，她们向时尚、追求时尚，但同时又内敛、
不夸张，她们是感性的且有一定的文化修养，在<script> 不经间流露出的是她们与 different 的气质。
衣着场所：休闲、聚会、购物、工作。
价格定位：春 夏 159-329 元之间，秋 冬 229-459 元之间。
产品结构：裙子，衣服，裤子，包包，饰品，鞋帽等多系列产品，中文名:沁园雪,类型:品牌名称,所属公司:东莞海自服饰有限公司</html_brief>


In [8]:
# Scientific Diagram Code Generation
# Generate TikZ code for recreating scientific diagrams
image_path = 'examples/example_diagram.png'
pixel_values = load_image(image_path, max_num=12).to(torch.bfloat16).cuda()
question = '<image>Please show me the TikZ code for displaying this image.'
answer = model.chat(tokenizer, pixel_values, question, generation_config, skip_special_tokens=False)
print('Answer:', answer)

Answer: <tikz>\documentclass[tikz,border=2mm]{standalone}
\usetikzlibrary{angles,quotes}

\begin{document}
\begin{tikzpicture}[scale=0.8]
\draw[thick] (0,0) coordinate (A) -- node[below] {$x$} (4,0) coordinate (B) -- node[above] {$y$} (0,2.5) coordinate (C) -- cycle;
\draw pic["$60^\circ$", draw, angle eccentricity=1.5, angle radius=0.8cm] {angle=B--C--A};
\draw (0,0.2) -- (0.2,0.2) -- (0.2,0);
\node[left] at (0,1.25) {5};
\end{tikzpicture}
\end{document}</tikz>


In [9]:
# Structured Data Extraction
# Extract structured information from ticket image
image_path = 'examples/example_ticket.jpg'
pixel_values = load_image(image_path, max_num=12).to(torch.bfloat16).cuda()
question = '<image>Get the text in the image as JSON.'
answer = model.chat(tokenizer, pixel_values, question, generation_config, skip_special_tokens=False)
print('Answer:', answer)

Answer: <json>{"出发站": "南京南站", "到达站": "北京南站", "座位类别": "一等座", "票价": "￥748.5元", "车票编码": "Z66N041840", "出发日期": "2017年11月16日", "车次": "G40", "乘客姓名": "嘉南"}</json>


In [10]:
# Extract textual content from chart image
image_path = 'examples/example_chart.png'
pixel_values = load_image(image_path, max_num=12).to(torch.bfloat16).cuda()
question = '<image>Use JSON to extract textual content from the image.'
answer = model.chat(tokenizer, pixel_values, question, generation_config, skip_special_tokens=False)
print('Answer:', answer)

Answer: <json>{"title": "Most preferred objects of different categories", "source": "None", "x_title": "None", "y_title": "Percent of People", "values": {"bird": {"car": "50 kg", "weight": "40 kg", "plain": "70 kg", "sir": "50 kg"}, "letter": {"car": "50 kg", "weight": "60 kg", "plain": "30 kg", "sir": "50 kg"}}}</json>
